In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets, Layout

# Boost Converter Plots
This notebook shows plots of the voltage across the capacitor (equivalent to $V_\mathrm{out}$) as well as the current through the inductor over multiple cycles of turning the transistor in the circuit on and off. This notebook allows you to interactively change the relevant values of the circuit to see how they affect the performance of the boost converter. It also shows the different regimes of the circuit (e.g., when the transistor and diode are on/off) via the dashed lines. **How does the inductor current curve in part (e) change if the battery voltage changes? What is the effect of a larger or smaller capacitor? A larger or smaller load?**

In [ ]:
x_init = np.array([3, 0]) # Initial conditions for V_c, I_L
V_bat = 1       # Volts
L = 1e-6        # Henrys
C = 1e-3        # Farads
R = 1e3         # Ohms
freq = 1.2e6      # Hz

def x_on(t, R, L, C, V_bat, x_init):
    return np.array([x_init[0] * np.exp(-t / (R * C)),
                     t * V_bat / L + x_init[1]])

def x_off_diode_on(t, R, L, C, V_bat, x_init):
    A_mat_off = np.array([[-1 / (R * C), 1 / C], [-1 / L, 0]])
    B_mat_off = np.array([0, V_bat / L])
    evals, V = np.linalg.eig(A_mat_off)
    V_inv = np.linalg.inv(V)

    z_init = V_inv @ x_init
    Bz = V_inv @ B_mat_off
    z1 = (z_init[0] + Bz[0] / evals[0]) * np.exp(t * evals[0]) - Bz[0] / evals[0] 
    z2 = (z_init[1] + Bz[1] / evals[1]) * np.exp(t * evals[1]) - Bz[1] / evals[1]
    return np.real(V @ np.array([z1, z2]))
    
def x_off_diode_off(t, R, C, x_init):
    return np.array([x_init[0] * np.exp(-t / (R * C)),
                     np.zeros(len(t))])

def generate_plot_data(num_cycles, num_samples, freq, R, L, C, V_bat, x_init):
    all_x = np.zeros((2, num_cycles * num_samples))
    all_t = np.linspace(0, num_cycles / freq, num=num_samples * num_cycles)
    new_x_init = x_init.copy()
    for i in range(num_cycles):
        t = np.linspace(0, 1 / freq, num=num_samples)

        # Starts with transistor on until inductor charges up to 100 mA at t_switch
        cycle_x = x_on(t, R, L, C, V_bat, new_x_init)
        thresh_vals = np.where(cycle_x[1] < 100e-3)[0]
        t_switch = thresh_vals.max() + 1 if thresh_vals.any() else 0

        # Then transistor turns off and we go until i_L = 0 at t_diode
        cycle_x[:, t_switch:] = x_off_diode_on(t, R, L, C, V_bat, cycle_x[:, t_switch])[:, :-t_switch]
        thresh_vals = np.where(cycle_x[1, t_switch:] > 0)[0]
        t_diode = t_switch + (thresh_vals.max() + 1 if thresh_vals.any() else 0)
        
        # Then inductor current is zero and V_c decays until the transistor turns back on
        cycle_x[:, t_diode:] = x_off_diode_off(t, R, C, cycle_x[:, t_diode])[:, :-t_diode]
        all_x[:, num_samples * i:num_samples * (i + 1)] = cycle_x
        new_x_init = cycle_x[:, -1]
    
    return all_t, all_x, t_switch, t_diode

# helper function to set the data for a vertical line
# t_offset is either 0, t_switch, or t_diode
def set_vline_data(vline, t_offset, freq):
    vline_segs = np.array(vline.get_segments())
    vline_segs[:, :, 0] = ((t_offset + np.arange(num_cycles) * num_samples) / (num_samples * freq))[:, None]
    vline.set_segments(vline_segs)

# Set up plots
num_cycles = 10 # Number of cycles to plot
num_samples = 1000 # Number of samples per cycle
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 8))
all_t, all_x, t_switch, t_diode = generate_plot_data(num_cycles, num_samples, freq, R, L, C, V_bat, x_init)
l1, = ax1.plot(all_t, all_x[0])
vbat_l, = ax1.plot(all_t, np.repeat(V_bat, len(all_t)), "r--")
ax1.set_xlim([0, all_t.max()])
ax1.set_ylim([0, 4])
ax1.set_xlabel("$t$ (s)")
ax1.set_ylabel("$V_C(t)$ (V)")
ax1.legend([l1, vbat_l], ["$V_C (t)$", "$V_\mathrm{bat}$"], loc="upper right")
ax1.set_title("$V_C(t)$ (on 0-4 V scale)")

l2, = ax2.plot(all_t, all_x[0])
ax2.set_xlim([0, all_t.max()])
ax2.set_xlabel("$t$ (s)")
ax2.set_ylim([3 - 3e-6, 3 + 3e-6])
ax2.set_ylabel("$V_C(t)$ (V)")
t_on_l1 = ax2.vlines((np.arange(num_cycles) * num_samples) / (num_samples * freq), 3 - 3e-6, 3 + 3e-6, colors="g", linestyles="dashed", linewidths=1)
t_off_l1 = ax2.vlines((t_switch + np.arange(num_cycles) * num_samples) / (num_samples * freq), 3 - 3e-6, 3 + 3e-6, colors="r", linestyles="dashed", linewidths=1)
d_off_l1 = ax2.vlines((t_diode + np.arange(num_cycles) * num_samples) / (num_samples * freq), 3 - 3e-6, 3 + 3e-6, colors="k", linestyles="dashed", linewidths=1)
ax2.legend([l1, t_on_l1, t_off_l1, d_off_l1], ["$V_C (t)$", "Transistor On, Diode Off", "Transistor Off, Diode On", "Transistor Off, Diode Off"], loc="upper right")
ax2.set_title("$V_C(t)$ (on 3 $\pm$ 3e-6 V scale)")

l3, = ax3.plot(all_t, all_x[1] * num_samples)
ax2_ylim = ax3.get_ylim()
t_on_l2 = ax3.vlines((np.arange(num_cycles) * num_samples) / (num_samples * freq), *ax2_ylim, colors="g", linestyles="dashed", linewidths=1)
t_off_l2 = ax3.vlines((t_switch + np.arange(num_cycles) * num_samples) / (num_samples * freq), *ax2_ylim, colors="r", linestyles="dashed", linewidths=1)
d_off_l2 = ax3.vlines((t_diode + np.arange(num_cycles) * num_samples) / (num_samples * freq), *ax2_ylim, colors="k", linestyles="dashed", linewidths=1)
ax3.set_xlim([0, all_t.max()])
ax3.set_ylim(ax2_ylim)
ax3.set_xlabel("$t$ (s)")
ax3.set_ylabel("$I_L(t)$ (mA)")
ax3.legend([l2, t_on_l2, t_off_l2, d_off_l2], ["$I_L (t)$", "Transistor On, Diode Off", "Transistor Off, Diode On", "Transistor Off, Diode Off"], loc="upper right")
ax3.set_title("$I_L(t)$")
fig.tight_layout()

# Main update loop
def update(freq=2e6, R=1e3, L=1e-6, C=1e-3, V_bat=1):
    all_t, all_x, t_switch, t_diode = generate_plot_data(num_cycles, num_samples, freq, R, L, C, V_bat, x_init)
    
    # Set data for curves
    l1.set_xdata(all_t)
    l1.set_ydata(all_x[0])
    vbat_l.set_xdata(all_t)
    vbat_l.set_ydata(np.repeat(V_bat, len(all_t)))
    l2.set_xdata(all_t)
    l2.set_ydata(all_x[0])
    l3.set_xdata(all_t)
    l3.set_ydata(all_x[1] * num_samples)
    ax1.set_xlim([0, all_t.max()])
    ax2.set_xlim([0, all_t.max()])
    ax3.set_xlim([0, all_t.max()])
    
    # Set data for vertical lines
    set_vline_data(t_on_l1, 0, freq)
    set_vline_data(t_on_l2, 0, freq)
    set_vline_data(t_off_l1, t_switch, freq)
    set_vline_data(t_off_l2, t_switch, freq)
    set_vline_data(d_off_l1, t_diode, freq)
    set_vline_data(d_off_l2, t_diode, freq)
    

# Create interactive sliders
freq_slider = widgets.FloatLogSlider(value=freq, min=np.log10(2e5), max=np.log10(3e6), step=.05, description='Frequency (Hz)', style={'description_width': 'initial'}, layout=Layout(width='100%'))
R_slider = widgets.FloatLogSlider(value=R, min=np.log10(500), max=np.log10(5000), step=.05, description='R ($\Omega$)', layout=Layout(width='100%'))
L_slider = widgets.FloatLogSlider(value=L, min=-7, max=np.log10(2e-6), step=.05, description='L (H)', layout=Layout(width='100%'))
C_slider = widgets.FloatLogSlider(value=C, min=-4, max=-2, step=.05, description='C (F)', layout=Layout(width='100%'))
V_bat_slider = widgets.FloatSlider(value=V_bat, min=0.8, max=2.1, step=0.1, description='$V_\mathrm{bat}$ (V)', layout=Layout(width='100%'))
interactive(update, freq=freq_slider, R=R_slider, L=L_slider, C=C_slider, V_bat=V_bat_slider)